<a href="https://colab.research.google.com/github/gebriell/kaggle_transformer/blob/main/colab_stacked_encoder_and_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = '140000:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4882008%2F8232040%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240502%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240502T084645Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D84a1fe89e81f68a52401576cb6d728648516dd823de3dc13bfb92097eb804ba697b33afdaeef83fccca56de0932babc426f9873ceeb6c60c4401d3a0002edd8eef7968943eaed5dcf0cc6afd66d99749386d670e54e2ec8a6a7499b350e773bb56d287e0082f321f857dd9ba208c3a121f183282e117b2cb2f6c5df22c5bda57a132ab3403822d0fb1039db81e1fdecf017a2d6a28b60fb803653da36f96644681f4eea30d6345fd95f83329d9d8f9fe05427fd2c177ad1559e083419886d8d28c645846929e588b0c2da87d18586b0bc3eff97f0fdbc65c87dbded1a2aecf7dbfd582576c44b4a1cc1358957652d97b099cf79d4cb503d6a1d67bebdaa83665'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 3982097426 bytes downloaded
Downloaded and uncompressed: 140000
Data source import complete.


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# For viewing and manipulating data
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Importing the necessary libraries

import re
import math
import torch
import spacy
import string
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import gensim.downloader as api
import matplotlib.pyplot as plt

# Getting particular functions from these libraries
from scipy import stats
from torch import Tensor
from sklearn.utils import resample
from gensim.models import KeyedVectors # >> alternative to gensim.downloader
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from torch.utils.data import random_split, DataLoader, TensorDataset, Dataset

# Using the NLTK to tokenize the text
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Checks if a CUDA enabled GPU is available and prints out its information
if torch.cuda.is_available():
    print("CUDA is available!")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

    device = torch.device("cuda:0")

else:
    print("CUDA is not available.")
    device = torch.device("cpu")

# !pip install torch_xla

# import torch_xla.core.xla_model as xm

# # Check if TPUs are available
# if xm.get_xla_supported_devices():
#     print("TPU is available!")
#     device = xm.xla_device()
# else:
#     print("TPU is not available. Falling back to CPU.")
#     device = torch.device("cpu")

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
reviews_file = ''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        file_name = os.path.join(dirname, filename)
        if file_name.endswith('.csv'):
            reviews_file = file_name
        elif file_name.endswith('.bin') or ('.gz'):
            w2v_file = file_name
        else:
            print(f'Found unexpected file: {file_name}')

print(f'Amazon reviews file: {reviews_file}')
print(f'Google news word to vec file: {w2v_file}')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
VERBOSE = False
def printv(text):
    if VERBOSE: print('VERBOSE:', text)
    return

def showV(text):
    '''unconditional verbose output'''
    print('VERBOSE:', text)
    return

DEV = True
def printd(text):
    if DEV: print('DEV:', text)
    return

def showD(text):
    '''unconditional DEV output'''
    print('VERBOSE:', text)
    return

showCellCompletion = False
def showC(text):
    if showCellCompletion:
        print('Cell complete:', text)
    return

import subprocess
showNv = False
accelerator = False

def printNv():
    if not showNv or not accelerator: return
    mem_usage = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE)
    print(mem_usage.stdout.decode('utf-8'))

showMemoryAllocation = True
def printM():
    if not showMemoryAllocation: return
    print(f"Total allocated memory: {torch.cuda.memory_allocated()} bytes")

CUDA is available!
GPU 0: NVIDIA A100-SXM4-40GB
Amazon reviews file: 
Google news word to vec file: /kaggle/input/140000/review_data.hdf5


In [ ]:
# import h5py

# file_path = '/kaggle/input/140000/review_data.hdf5'
# with h5py.File(file_path, 'r') as hf:
#     # Access the datasets within the HDF5 file
#     text_reviews_dataset = hf['text_reviews']
#     ratings_dataset = hf['ratings']

#     # Convert the datasets to PyTorch tensors
#     text_reviews = torch.from_numpy(text_reviews_dataset[:])
#     ratings = torch.from_numpy(ratings_dataset[:])

# # check loaded tensors
# print(text_reviews.shape)
# print(ratings.shape)

# class ReviewDataset(Dataset):
#     def __init__(self, text_reviews, ratings):
#         self.text_reviews = text_reviews
#         self.ratings = ratings

#     def __len__(self):
#         return len(self.ratings)

#     def __getitem__(self, index):
#         review = self.text_reviews[index]
#         rating = self.ratings[index]
#         return review, rating

# # Create the dataset
# dataset = ReviewDataset(text_reviews, ratings)

# # Create data loaders
# data_loader = DataLoader(dataset, batch_size = 32, shuffle = False)

# Hyperparameters

In [4]:
import h5py

file_path = '/kaggle/input/140000/review_data.hdf5'
with h5py.File(file_path, 'r') as hf:
    # Access the datasets within the HDF5 file
    text_reviews_dataset = hf['text_reviews']
    ratings_dataset = hf['ratings']

    # Convert the datasets to PyTorch tensors
    text_reviews = torch.from_numpy(text_reviews_dataset[:])
    ratings = torch.from_numpy(ratings_dataset[:])

# check loaded tensors
print(text_reviews.shape)
print(ratings.shape)

# # Create the dataset
dataset = TensorDataset(text_reviews, ratings)

# DataLoader for training data
data_loader = DataLoader(dataset, batch_size = 32, shuffle = False)

torch.Size([140000, 100, 300])
torch.Size([140000])


In [5]:
# HyperParameters for the module
d_model = 300  # Should match the embedding dimension of our word embeddings
seq_len = 100  # Maximum sequence length
input_size = d_model  # based on the output size of our feed-forward network

num_layers = 6 # Number of encoder layers
h       = 30 # number of attention head
d_ffn   = 2048 # dimension of the feedforward layer

dropout = 0.0 # we can adjust the dropout if needed

"""
To ensure compatibility, it's important to choose the
number of attention heads (h) such that d_model is
evenly divisible by h in the multi-head attention
module's self.d_k. This allows for a clean distribution
of the model dimensionality across the attention heads.
"""

# d_model / attn.h = 300 / 30 = 10

"\nTo ensure compatibility, it's important to choose the \nnumber of attention heads (h) such that d_model is \nevenly divisible by h in the multi-head attention \nmodule's self.d_k. This allows for a clean distribution \nof the model dimensionality across the attention heads.\n"

# Pytorch's Transformer Encoder


In [6]:
pytorch_encoder_layer = nn.TransformerEncoderLayer(d_model,
                                                   h, d_ffn, dropout).to(device)

pytorch_encoded_reviews = []  # List to store encoded batches
pytorch_ratings = []        # List to store corresponding y batches

i = 0

with torch.no_grad():
    for reviews, ratings in data_loader:
        print(f'Size of batch: {reviews.shape}')
        i += 1
        print(i)

        # Move the batch to the device
        reviews = reviews.to(device)

        output_features = pytorch_encoder_layer(reviews)

        # Move the encoded data back to the CPU
        # output_features = output_features.cpu()

        # Append the encoded batch to the list
        pytorch_encoded_batches.append(output_features)
        pytorch_ratings.append(ratings)

# Concatenate all the encoded batches into a single tensor
pytorch_encoded_batch = torch.cat(pytorch_encoded_batches, dim = 0)

# Concatenate all the corresponding y batches into a single tensor
pytorch_ratings = torch.cat(pytorch_ratings, dim = 0)

print(f'Size of Reviews: {len(pytorch_encoded_batches)} tensors')
print(f'Size of Ratings: {len(pytorch_ratings)} tensors')
print(pytorch_encoded_reviews.shape)

Size of batch: torch.Size([32, 100, 300])
1


NameError: name 'pytorch_encoded_batches' is not defined

In [ ]:
import h5py

# Open the HDF5 file in write mode ('w')
with h5py.File('transformer_data.hdf5', 'w') as hf:

    # Create separate datasets for text reviews and ratings within the HDF5 file
    hf.create_dataset('text_reviews', data = pytorch_encoded_reviews.cpu().numpy())
    hf.create_dataset('ratings', data = pytorch_ratings.cpu().numpy())

# Pytorch's Stacked Transformer Encoder


In [7]:
encoder_layer = nn.TransformerEncoderLayer(d_model, h, d_ffn, dropout).to(device)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers).to(device)

encoded_reviews = []
corresponding_ratings = []

batch_counter = 0

with torch.no_grad():
    for reviews, ratings in data_loader:
        print(f'Size of batch: {reviews.shape}')
        batch_counter += 1
        print(batch_counter)

        reviews = reviews.to(device)
        encoded_batch = encoder_layer(reviews)
        encoded_batch = encoded_batch.cpu()  # Move the encoded batch to CPU
        encoded_reviews.append(encoded_batch)
        corresponding_ratings.append(ratings.cpu())  # Move the ratings to CPU as well

encoded_reviews_tensor = torch.cat(encoded_reviews, dim=0)
ratings_tensor = torch.cat(corresponding_ratings, dim=0)

print(f'Size of Reviews: {len(encoded_reviews_tensor)} tensors')
print(f'Size of Ratings: {len(ratings_tensor)} tensors')
print(encoded_reviews_tensor.shape)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Streaming output truncated to the last 5000 lines.
1877
Size of batch: torch.Size([32, 100, 300])
1878
Size of batch: torch.Size([32, 100, 300])
1879
Size of batch: torch.Size([32, 100, 300])
1880
Size of batch: torch.Size([32, 100, 300])
1881
Size of batch: torch.Size([32, 100, 300])
1882
Size of batch: torch.Size([32, 100, 300])
1883
Size of batch: torch.Size([32, 100, 300])
1884
Size of batch: torch.Size([32, 100, 300])
1885
Size of batch: torch.Size([32, 100, 300])
1886
Size of batch: torch.Size([32, 100, 300])
1887
Size of batch: torch.Size([32, 100, 300])
1888
Size of batch: torch.Size([32, 100, 300])
1889
Size of batch: torch.Size([32, 100, 300])
1890
Size of batch: torch.Size([32, 100, 300])
1891
Size of batch: torch.Size([32, 100, 300])
1892
Size of batch: torch.Size([32, 100, 300])
1893
Size of batch: torch.Size([32, 100, 300])
1894
Size of batch: torch.Size([32, 100, 300])
1895
Size of batch: torch.Size([32, 100, 300])
1896
Size of batch: torch.Size([32, 100, 300])
1897
Size

In [8]:
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
from collections import Counter

class ReviewDataset(Dataset):
    def __init__(self, reviews, ratings):
        self.reviews = reviews
        self.ratings = ratings

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, index):
        review = self.reviews[index]
        rating = self.ratings[index]
        return review, rating

dataset = ReviewDataset(encoded_reviews_tensor, ratings_tensor)

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_indices, val_indices = next(sss.split(range(len(dataset)), dataset.ratings))

train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(len(train_indices))
print(len(val_indices))

print("Training Set:")
print("Number of reviews:", len(train_dataset))
print("Number of reviews per rating:")
train_ratings = [rating.item() for _, rating in train_dataset]
train_rating_counts = Counter(train_ratings)
for rating, count in sorted(train_rating_counts.items()):
    print(f"Rating {rating}: {count} reviews")

print("\nValidation Set:")
print("Number of reviews:", len(val_dataset))
print("Number of reviews per rating:")
val_ratings = [rating.item() for _, rating in val_dataset]
val_rating_counts = Counter(val_ratings)
for rating, count in sorted(val_rating_counts.items()):
    print(f"Rating {rating}: {count} reviews")

assert len(train_dataset) == 112000, f"Expected 112000 training reviews, got {len(train_dataset)}"
assert len(val_dataset) == 28000, f"Expected 28000 validation reviews, got {len(val_dataset)}"

for rating in sorted(train_rating_counts.keys()):
    train_count = train_rating_counts[rating]
    val_count = val_rating_counts[rating]
    assert train_count == val_count * 4, f"Rating {rating}: Expected 4 times more reviews in training set than validation set"

# HyperParameters for the model
d_model = 300  # Should match the embedding dimension of your word embeddings
seq_len = 100 #<< 4/13/24 100  # Maximum sequence length
dropout = 0.1  # Adjust the dropout if needed

num_layers = 25 # depth of our network
input_size = d_model  # match the output dim of your ff_net
num_classes = 5  # our ratings (1 - 5)
hidden_size = 2048 # 2^n

eps    = 1e-05 # epsilon value to prevent the standard deviation from becoming zero
epochs = 200 #<< 100
learning_rate = 0.001
weight_decay  = 0.01

showC('Hyperparameters defined')

class NeuralNetClassifier(nn.Module):
    def __init__(self, r_size, v_size, num_classes, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout):
        super(NeuralNetClassifier, self).__init__()

        self.hidden_layers = nn.ModuleList()
        self.hidden_layers.append(nn.Linear(r_size * v_size, hidden_size))
        self.hidden_layers.append(nn.BatchNorm1d(hidden_size))

        for _ in range(num_layers - 1):
            self.hidden_layers.append(nn.Linear(hidden_size, hidden_size))
            self.hidden_layers.append(nn.BatchNorm1d(hidden_size))

        self.output_layer = nn.Linear(hidden_size, num_classes)
        self.relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        for layer in self.hidden_layers:
            x = layer(x)
            x = self.relu(x)
            x = self.dropout(x)

        x = self.output_layer(x)
        return x

classifier = NeuralNetClassifier(seq_len, d_model, num_classes, hidden_size, num_layers, dropout)
classifier = classifier.to(device)

print(classifier)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(classifier.parameters(), lr=learning_rate, weight_decay=weight_decay)

DEV = True

losses = {}
for epoch in range(epochs):
    for inputs, targets in train_loader:
        optimizer.zero_grad()

        inputs = inputs.to(device)  # Move inputs to GPU
        targets = targets.to(device) - 1
        inputs = torch.reshape(inputs, (inputs.size(0), -1))

        outputs = classifier(inputs)

        loss = criterion(outputs, targets.long())
        loss.backward(retain_graph=True)

        optimizer.step()

        losses[loss.item()] = epoch + 1
        printv(f'Epoch [{epoch + 1} / {epochs}] Loss: {loss.item()}')

if VERBOSE:
    printv(f'Last loss: Epoch [{epoch + 1} / {epochs}] Loss: {loss.item()}')
    smallest_losses = sorted(list(losses.keys()))
    printv('Smallest losses')
    for idx in range(3):
        l = smallest_losses[idx]
        printv(f'    Loss: {l}, epoch = {losses[l]}')
showC(f'training complete')

# Put model in evaluation mode
classifier.eval()

# Tracking variables
predictions = []
actuals = []

# Evaluate on validation set
with torch.no_grad():
    for inputs, targets in val_loader:
        inputs = inputs.reshape(inputs.shape[0], -1).to(device)
        targets = targets.to(device) - 1  # Convert ratings from [1, 5] to [0, 4]

        outputs = classifier(inputs)
        _, predicted = torch.max(outputs, 1)

        predictions.extend(predicted.tolist())
        actuals.extend(targets.tolist())

# Calculate evaluation metrics
accuracy = accuracy_score(actuals, predictions)
precision = precision_score(actuals, predictions, average='weighted')
recall = recall_score(actuals, predictions, average='weighted')
f1 = f1_score(actuals, predictions, average='weighted')

# Print evaluation metrics
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation Precision: {precision:.4f}")
print(f"Validation Recall: {recall:.4f}")
print(f"Validation F1-score: {f1:.4f}")

# Calculate confusion matrix
cm = confusion_matrix(actuals, predictions)
print("Confusion Matrix:")
print(cm)

# Assess bias and variance
if accuracy < 0.7:  # Adjust the threshold as per your requirements
    print("The model may have high bias (underfitting). Consider increasing model complexity.")
elif accuracy > 0.95:  # Adjust the threshold as per your requirements
    print("The model may have high variance (overfitting). Consider regularization techniques.")
else:
    print("The model seems to have a good balance between bias and variance.")

112000
28000
Training Set:
Number of reviews: 112000
Number of reviews per rating:
Rating 1: 22400 reviews
Rating 2: 22400 reviews
Rating 3: 22400 reviews
Rating 4: 22400 reviews
Rating 5: 22400 reviews

Validation Set:
Number of reviews: 28000
Number of reviews per rating:
Rating 1: 5600 reviews
Rating 2: 5600 reviews
Rating 3: 5600 reviews
Rating 4: 5600 reviews
Rating 5: 5600 reviews
NeuralNetClassifier(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=30000, out_features=2048, bias=True)
    (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Linear(in_features=2048, out_features=2048, bias=True)
    (3): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=2048, out_features=2048, bias=True)
    (5): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=2048, out_features=2048, bias=True)
    (7): BatchNorm1d(2048, e